# Installation 

In [87]:
import pandas as pd
import numpy as np
import requests
import time
import json
import re

In [88]:
API_KEY = '39Lmr8g-EvTQ_EuO1ZnoNtri4K5NXh9cBB4zUl6vpV0' 

# Merging Datasets

In [89]:
zhvi = pd.read_csv('zhvi_county.csv')
df= zhvi

date_columns = [col for col in df.columns if '-' in col]
years = [col.split('-')[0] for col in date_columns]
year_to_columns = {year: [] for year in set(years)}

for col, year in zip(date_columns, years):
    year_to_columns[year].append(col)
    
df_avg = df.drop(columns=date_columns).copy()

for year, cols in year_to_columns.items():
    df_avg[year] = df[cols].mean(axis=1, skipna=True)


df_avg_sorted = df_avg.reindex(sorted(df_avg.columns), axis=1)
df_avg_sorted_filtered = df_avg_sorted.drop(columns=['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2020', '2023', '2024'])
df_avg_sorted_filtered

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,...,2022,Metro,MunicipalCodeFIPS,RegionID,RegionName,RegionType,SizeRank,State,StateCodeFIPS,StateName
0,391800.758750,370021.247571,359359.781033,413367.526346,467421.730158,493329.506511,512422.260226,556816.255058,621041.875761,643540.730462,...,865028.101827,"Los Angeles-Long Beach-Anaheim, CA",37,3101,Los Angeles County,county,0,CA,6,CA
1,195895.506753,175997.391212,160009.360039,167472.384138,187767.400216,197564.055315,209375.472283,220579.787072,231962.813492,235328.661939,...,285327.173523,"Chicago-Naperville-Elgin, IL-IN-WI",31,139,Cook County,county,1,IL,17,IL
2,133684.697930,127473.166932,125860.876114,133201.326267,149536.640099,166208.908352,177454.225391,185192.688212,193903.462329,202957.819887,...,280315.112397,"Houston-The Woodlands-Sugar Land, TX",201,1090,Harris County,county,2,TX,48,TX
3,158523.953547,140071.996551,152406.961403,188059.831584,209946.586531,221067.229845,240703.953236,255982.961388,275284.116113,292423.168507,...,483175.892678,"Phoenix-Mesa-Chandler, AZ",13,2402,Maricopa County,county,3,AZ,4,AZ
4,374702.662441,354421.569709,351974.029262,407181.803531,455299.811442,487252.407992,525004.500223,563629.359331,599028.850019,608870.171560,...,883859.454374,"San Diego-Chula Vista-Carlsbad, CA",73,2841,San Diego County,county,4,CA,6,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,312376.580708,"Scottsbluff, NE",7,846,Banner County,county,3206,NE,31,NE
3070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,9,1648,Daggett County,county,3207,UT,49,UT
3071,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,171,1432,Thomas County,county,3208,NE,31,NE
3072,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,274459.913989,"North Platte, NE",117,2794,McPherson County,county,3212,NE,31,NE


In [90]:
variables_definations_for_B17017 = {
    "B17017_002E":  "Income in the past 12 months below poverty level",
    "B17017_031E": "Income in the past 12 months at or above poverty level"
}

index_for_variables_B17017 = {
    "B17017_002E": 4,
    "B17017_031E": 120,
    "GEO_ID" : 236,
    "NAME" : 237
}

index_for_variables_B17017_2010_2011 = {
    "B17017_002E": 3,
    "B17017_031E": 61,
    "GEO_ID" : 0,
    "NAME" : 119
}

In [91]:
B17017_data_files = ['B17017_Data_2010.json', 'B17017_Data_2011.json', 'B17017_Data_2012.json', 'B17017_Data_2013.json', 'B17017_Data_2014.json', 'B17017_Data_2015.json', 'B17017_Data_2016.json', 'B17017_Data_2017.json', 'B17017_Data_2018.json', 'B17017_Data_2019.json', 'B17017_Data_2021.json','B17017_Data_2022.json']
B17017_data = {}
for i in range(len(B17017_data_files)):
    B17017_json = json.load(open(B17017_data_files[i]))
    temp = []

    for var in B17017_json:
        if(B17017_data_files[i] == 'B17017_Data_2010.json' or B17017_data_files[i] == 'B17017_Data_2011.json'):
            temp.append([var[index_for_variables_B17017_2010_2011['GEO_ID']], 
                     var[index_for_variables_B17017_2010_2011['NAME']], 
                      var[index_for_variables_B17017_2010_2011['B17017_002E']],
                       var[index_for_variables_B17017_2010_2011['B17017_031E']]])
        else:
          temp.append([var[index_for_variables_B17017['GEO_ID']], 
                     var[index_for_variables_B17017['NAME']], 
                      var[index_for_variables_B17017['B17017_002E']],
                       var[index_for_variables_B17017['B17017_031E']]])
    
    B17017_data[re.search('Data_(.+?).json', B17017_data_files[i]).group(1)] = temp

In [92]:
B17017_data['2010'][0]

['GEO_ID', 'NAME', 'B17017_002E', 'B17017_031E']

In [93]:
B17017_data['2010'][1]

['0500000US39151', 'Stark County, Ohio', '18744', '130378']

In [94]:
variables_definations_for_S2502 = {
    "S2502_C01_001E" :  "Total Occupied Housing Units",
    "S2502_C01_008E" : "two or more races",
    "S2502_C01_009E" : "Hispanic or Latino origin",
    "S2502_C01_010E" : "White alone, not Hispanic or Latino",
    "S2502_C01_011E" : "under 35 years old",
    "S2502_C01_012E" : "35 to 44 years old",
    "S2502_C01_013E" : "45 to 54 years old",
    "S2502_C01_014E" : "55 to 64 years old",
    "S2502_C01_015E" : "65 to 74 years old",
    "S2502_C01_016E" : "75 to 84 years old",
    "S2502_C01_017E" : "85 years and over",
    "S2502_C01_018E" : "Less than high school graduate",
    "S2502_C01_019E" : "High school graduate (includes equivalency)",
    "S2502_C01_020E" : "Some college or associate's degree",
    "S2502_C01_021E" : "Bachelor's degree or higher",
}

index_for_variables_S2502 = {
    "S2502_C01_001E" : 2,
    "S2502_C01_008E" : 30,
    "S2502_C01_009E" : 34,
    "S2502_C01_010E" : 38,
    "S2502_C01_011E" : 42,
    "S2502_C01_012E" : 46,
    "S2502_C01_013E" : 50,
    "S2502_C01_014E" : 54,
    "S2502_C01_015E" : 58,
    "S2502_C01_016E" : 62,
    "S2502_C01_017E" : 66,
    "S2502_C01_018E" : 70,
    "S2502_C01_019E" : 74,
    "S2502_C01_020E" : 78,
    "S2502_C01_021E" : 82,
    "GEO_ID" : 0,
    "NAME" : 1
}

index_for_variables_S2502_2010_2011 = {
    "S2502_C01_001E" : 1,
    "S2502_C01_008E" : 15,
    "S2502_C01_009E" : 17,
    "S2502_C01_010E" : 19,
    "S2502_C01_011E" : 21,
    "S2502_C01_012E" : 23,
    "S2502_C01_013E" : 25,
    "S2502_C01_014E" : 27,
    "S2502_C01_015E" : 29,
    "S2502_C01_016E" : 31,
    "S2502_C01_017E" : 33,
    "S2502_C01_018E" : 35,
    "S2502_C01_019E" : 37,
    "S2502_C01_020E" : 39,
    "S2502_C01_021E" : 41,
    "GEO_ID" : 0,
    "NAME" : 169
}

In [95]:
S2502_data_files = ['S2502_Data_2010.json', 'S2502_Data_2011.json', 'S2502_Data_2012.json', 'S2502_Data_2013.json', 'S2502_Data_2014.json', 'S2502_Data_2015.json', 'S2502_Data_2016.json', 'S2502_Data_2017.json', 'S2502_Data_2018.json', 'S2502_Data_2019.json', 'S2502_Data_2021.json','S2502_Data_2022.json']
S2502_data = {}
for i in range(len(S2502_data_files)):
    S2502_json = json.load(open(S2502_data_files[i]))
    temp = []

    for var in S2502_json:
        if(S2502_data_files[i] == 'S2502_Data_2010.json' or S2502_data_files[i] == 'S2502_Data_2011.json'):
            if(var[0] != "GEO_ID"):
              temp.append([var[index_for_variables_S2502_2010_2011['GEO_ID']],
                      var[index_for_variables_S2502_2010_2011['NAME']],
                      var[index_for_variables_S2502_2010_2011['S2502_C01_001E']],
                          round(0.01 * float(var[index_for_variables_S2502_2010_2011['S2502_C01_008E']]) * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']])) if (var[index_for_variables_S2502_2010_2011['S2502_C01_008E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_008E']]) > 0 ) else 0,
                            round(0.01 * float(var[index_for_variables_S2502_2010_2011['S2502_C01_009E']]) * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']])) if (var[index_for_variables_S2502_2010_2011['S2502_C01_009E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_009E']]) > 0 ) else 0,
                              round(0.01 * float(var[index_for_variables_S2502_2010_2011['S2502_C01_010E']]) * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']])) if (var[index_for_variables_S2502_2010_2011['S2502_C01_010E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_010E']]) > 0 ) else 0,
                                round(0.01 * float(var[index_for_variables_S2502_2010_2011['S2502_C01_011E']]) * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']])) if (var[index_for_variables_S2502_2010_2011['S2502_C01_011E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_011E']]) > 0 ) else 0,
                                  round(0.01 * float(var[index_for_variables_S2502_2010_2011['S2502_C01_012E']]) * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']])) if (var[index_for_variables_S2502_2010_2011['S2502_C01_012E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_012E']]) > 0 ) else 0,
                                    round(0.01 * float(var[index_for_variables_S2502_2010_2011['S2502_C01_013E']]) * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']])) if (var[index_for_variables_S2502_2010_2011['S2502_C01_013E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_013E']]) > 0 ) else 0,
                                      round(0.01 * float(var[index_for_variables_S2502_2010_2011['S2502_C01_014E']]) * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']])) if (var[index_for_variables_S2502_2010_2011['S2502_C01_014E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_014E']]) > 0 ) else 0,
                                        round(0.01 * float(var[index_for_variables_S2502_2010_2011['S2502_C01_015E']]) * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']])) if (var[index_for_variables_S2502_2010_2011['S2502_C01_015E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_015E']]) > 0 ) else 0,
                                          round(0.01 * float(var[index_for_variables_S2502_2010_2011['S2502_C01_016E']]) * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']])) if (var[index_for_variables_S2502_2010_2011['S2502_C01_016E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_016E']]) > 0 ) else 0,
                                            round(0.01 * float(var[index_for_variables_S2502_2010_2011['S2502_C01_017E']]) * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']])) if (var[index_for_variables_S2502_2010_2011['S2502_C01_017E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_017E']]) > 0 ) else 0,
                                              round(0.01 * float(var[index_for_variables_S2502_2010_2011['S2502_C01_018E']]) * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']])) if (var[index_for_variables_S2502_2010_2011['S2502_C01_018E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_018E']]) > 0 ) else 0,
                                                round(0.01 * float(var[index_for_variables_S2502_2010_2011['S2502_C01_019E']]) * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']])) if (var[index_for_variables_S2502_2010_2011['S2502_C01_019E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_019E']]) > 0 ) else 0,
                                                  round(0.01 * float(var[index_for_variables_S2502_2010_2011['S2502_C01_020E']]) * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']])) if (var[index_for_variables_S2502_2010_2011['S2502_C01_020E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_020E']]) > 0 ) else 0,
                                                    round(0.01 * float(var[index_for_variables_S2502_2010_2011['S2502_C01_021E']]) * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']])) if (var[index_for_variables_S2502_2010_2011['S2502_C01_021E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_021E']]) > 0 ) else 0,
                                                     round(float(100 - float(var[index_for_variables_S2502_2010_2011['S2502_C01_011E']]) if (var[index_for_variables_S2502_2010_2011['S2502_C01_011E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_011E']]) > 0) else 0) * 0.01 * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']]))])
            else:
                  temp.append([var[index_for_variables_S2502_2010_2011['GEO_ID']],
                      var[index_for_variables_S2502_2010_2011['NAME']],
                        var[index_for_variables_S2502_2010_2011['S2502_C01_001E']],
                          var[index_for_variables_S2502_2010_2011['S2502_C01_008E']],
                            var[index_for_variables_S2502_2010_2011['S2502_C01_009E']],
                              var[index_for_variables_S2502_2010_2011['S2502_C01_010E']],
                                var[index_for_variables_S2502_2010_2011['S2502_C01_011E']],
                                  var[index_for_variables_S2502_2010_2011['S2502_C01_012E']],
                                    var[index_for_variables_S2502_2010_2011['S2502_C01_013E']],
                                      var[index_for_variables_S2502_2010_2011['S2502_C01_014E']],
                                        var[index_for_variables_S2502_2010_2011['S2502_C01_015E']],
                                          var[index_for_variables_S2502_2010_2011['S2502_C01_016E']],
                                            var[index_for_variables_S2502_2010_2011['S2502_C01_017E']],
                                              var[index_for_variables_S2502_2010_2011['S2502_C01_018E']],
                                                var[index_for_variables_S2502_2010_2011['S2502_C01_019E']],
                                                  var[index_for_variables_S2502_2010_2011['S2502_C01_020E']],
                                                    var[index_for_variables_S2502_2010_2011['S2502_C01_021E']],
                                                      "Above 35 years old"])
                          
        elif(S2502_data_files[i] in ['S2502_Data_2012.json', 'S2502_Data_2013.json', 'S2502_Data_2014.json', 'S2502_Data_2015.json', 'S2502_Data_2016.json']):
            if (var[0] != "GEO_ID"): 
              temp.append([var[index_for_variables_S2502['GEO_ID']],
                     var[index_for_variables_S2502['NAME']],
                       var[index_for_variables_S2502['S2502_C01_001E']] if (var[index_for_variables_S2502['S2502_C01_001E']] != None) else 0,
                          round(0.01 * float(var[index_for_variables_S2502['S2502_C01_008E']]) * int(var[index_for_variables_S2502['S2502_C01_001E']])) if (var[index_for_variables_S2502['S2502_C01_008E']] != None and float(var[index_for_variables_S2502['S2502_C01_008E']]) > 0) else 0,
                            round(0.01 * float(var[index_for_variables_S2502['S2502_C01_009E']]) * int(var[index_for_variables_S2502['S2502_C01_001E']])) if (var[index_for_variables_S2502['S2502_C01_009E']] != None and float(var[index_for_variables_S2502['S2502_C01_009E']]) > 0) else 0,
                              round(0.01 * float(var[index_for_variables_S2502['S2502_C01_010E']]) * int(var[index_for_variables_S2502['S2502_C01_001E']])) if (var[index_for_variables_S2502['S2502_C01_010E']] != None and float(var[index_for_variables_S2502['S2502_C01_010E']]) > 0) else 0,
                                round(0.01 * float(var[index_for_variables_S2502['S2502_C01_011E']]) * int(var[index_for_variables_S2502['S2502_C01_001E']])) if (var[index_for_variables_S2502['S2502_C01_011E']] != None and float(var[index_for_variables_S2502['S2502_C01_011E']]) > 0) else 0,
                                  round(0.01 * float(var[index_for_variables_S2502['S2502_C01_012E']]) * int(var[index_for_variables_S2502['S2502_C01_001E']])) if (var[index_for_variables_S2502['S2502_C01_012E']] != None) and float(var[index_for_variables_S2502['S2502_C01_012E']]) > 0 else 0, 
                                    round(0.01 * float(var[index_for_variables_S2502['S2502_C01_013E']]) * int(var[index_for_variables_S2502['S2502_C01_001E']])) if (var[index_for_variables_S2502['S2502_C01_013E']] != None and float(var[index_for_variables_S2502['S2502_C01_013E']]) > 0) else 0,
                                      round(0.01 * float(var[index_for_variables_S2502['S2502_C01_014E']]) * int(var[index_for_variables_S2502['S2502_C01_001E']])) if (var[index_for_variables_S2502['S2502_C01_014E']] != None and float(var[index_for_variables_S2502['S2502_C01_014E']]) > 0) else 0,
                                        round(0.01 * float(var[index_for_variables_S2502['S2502_C01_015E']]) * int(var[index_for_variables_S2502['S2502_C01_001E']])) if (var[index_for_variables_S2502['S2502_C01_015E']] != None and float(var[index_for_variables_S2502['S2502_C01_015E']]) > 0) else 0,
                                          round(0.01 * float(var[index_for_variables_S2502['S2502_C01_016E']]) * int(var[index_for_variables_S2502['S2502_C01_001E']])) if (var[index_for_variables_S2502['S2502_C01_016E']] != None and float(var[index_for_variables_S2502['S2502_C01_016E']]) > 0) else 0,
                                            round(0.01 * float(var[index_for_variables_S2502['S2502_C01_017E']]) * int(var[index_for_variables_S2502['S2502_C01_001E']])) if (var[index_for_variables_S2502['S2502_C01_017E']] != None and float(var[index_for_variables_S2502['S2502_C01_017E']]) > 0) else 0,
                                              round(0.01 * float(var[index_for_variables_S2502['S2502_C01_018E']]) * int(var[index_for_variables_S2502['S2502_C01_001E']])) if (var[index_for_variables_S2502['S2502_C01_018E']] != None and float(var[index_for_variables_S2502['S2502_C01_018E']]) > 0) else 0,
                                                round(0.01 * float(var[index_for_variables_S2502['S2502_C01_019E']]) * int(var[index_for_variables_S2502['S2502_C01_001E']])) if (var[index_for_variables_S2502['S2502_C01_019E']] != None and float(var[index_for_variables_S2502['S2502_C01_019E']]) > 0) else 0,
                                                  round(0.01 * float(var[index_for_variables_S2502['S2502_C01_020E']]) * int(var[index_for_variables_S2502['S2502_C01_001E']])) if (var[index_for_variables_S2502['S2502_C01_020E']] != None and float(var[index_for_variables_S2502['S2502_C01_020E']]) > 0) else 0,
                                                    round(0.01 * float(var[index_for_variables_S2502['S2502_C01_021E']]) * int(var[index_for_variables_S2502['S2502_C01_001E']])) if (var[index_for_variables_S2502['S2502_C01_021E']] != None and float(var[index_for_variables_S2502['S2502_C01_021E']]) > 0) else 0,
                                                      round(float(100 - float(var[index_for_variables_S2502['S2502_C01_011E']]) if (var[index_for_variables_S2502['S2502_C01_011E']] != None and float(var[index_for_variables_S2502['S2502_C01_011E']]) > 0) else 0) * 0.01 * int(var[index_for_variables_S2502['S2502_C01_001E']]))])
            else:
                  temp.append([var[index_for_variables_S2502['GEO_ID']],
                      var[index_for_variables_S2502['NAME']],
                        var[index_for_variables_S2502['S2502_C01_001E']],
                          var[index_for_variables_S2502['S2502_C01_008E']],
                            var[index_for_variables_S2502['S2502_C01_009E']],
                              var[index_for_variables_S2502['S2502_C01_010E']],
                                var[index_for_variables_S2502['S2502_C01_011E']],
                                  var[index_for_variables_S2502['S2502_C01_012E']],
                                    var[index_for_variables_S2502['S2502_C01_013E']],
                                      var[index_for_variables_S2502['S2502_C01_014E']],
                                        var[index_for_variables_S2502['S2502_C01_015E']],
                                          var[index_for_variables_S2502['S2502_C01_016E']],
                                            var[index_for_variables_S2502['S2502_C01_017E']],
                                              var[index_for_variables_S2502['S2502_C01_018E']],
                                                var[index_for_variables_S2502['S2502_C01_019E']],
                                                  var[index_for_variables_S2502['S2502_C01_020E']],
                                                    var[index_for_variables_S2502['S2502_C01_021E']],
                                                      "Above 35 years old"])
    
        else:
            temp.append([var[index_for_variables_S2502['GEO_ID']],
                     var[index_for_variables_S2502['NAME']],
                       var[index_for_variables_S2502['S2502_C01_001E']],
                         var[index_for_variables_S2502['S2502_C01_008E']],
                           var[index_for_variables_S2502['S2502_C01_009E']],
                             var[index_for_variables_S2502['S2502_C01_010E']],
                               var[index_for_variables_S2502['S2502_C01_011E']],
                                 var[index_for_variables_S2502['S2502_C01_012E']],
                                   var[index_for_variables_S2502['S2502_C01_013E']],
                                     var[index_for_variables_S2502['S2502_C01_014E']],
                                       var[index_for_variables_S2502['S2502_C01_015E']],
                                         var[index_for_variables_S2502['S2502_C01_016E']],
                                           var[index_for_variables_S2502['S2502_C01_017E']],
                                             var[index_for_variables_S2502['S2502_C01_018E']],
                                               var[index_for_variables_S2502['S2502_C01_019E']],
                                                 var[index_for_variables_S2502['S2502_C01_020E']],
                                                   var[index_for_variables_S2502['S2502_C01_021E']],
                                                    int(var[index_for_variables_S2502['S2502_C01_001E']]) - int(var[index_for_variables_S2502['S2502_C01_011E']] if (var[index_for_variables_S2502['S2502_C01_011E']] != None) else 0)  if (var[0] != "GEO_ID") else "Above 35 years old"] )
    
    S2502_data[re.search('Data_(.+?).json', S2502_data_files[i]).group(1)] = temp
    

In [96]:
variables_definations_for_S2503 = {
    "S2503_C01_001E" :  "Total Occupied Housing Units",
    "S2503_C01_026E" : "Less than $20,000,Less than 20 percent",
    "S2503_C01_028E" : "Less than $20,000,30 percent or more",
    "S2503_C01_030E" : "$20,000 to $34,999,Less than 20 percent",
    "S2503_C01_032E" : "$20,000 to $34,999,30 percent or more",
    "S2503_C01_034E" : "$35,000 to $49,999,Less than 20 percent",
    "S2503_C01_036E" : "$35,000 to $49,999,30 percent or more",
    "S2503_C01_038E" : "$50,000 to $74,999,Less than 20 percent",
    "S2503_C01_040E" : "$50,000 to $74,999,30 percent or more",
    "S2503_C01_042E" :  "$75,000 or more!!Less than 20 percent",
    "S2503_C01_044E" :  "$75,000 or more!!30 percent or more"
}

index_for_variables_S2503 = {
    "S2503_C01_001E" : 2,
    "S2503_C01_026E" : 102,
    "S2503_C01_028E" : 110,
    "S2503_C01_030E" : 118,
    "S2503_C01_032E" : 126,
    "S2503_C01_034E" : 134,
    "S2503_C01_036E" : 142,
    "S2503_C01_038E" : 150,
    "S2503_C01_040E" : 158,
    "S2503_C01_042E" : 166,
    "S2503_C01_044E" : 174,
    "GEO_ID" : 0,
    "NAME" : 1
}

index_for_variables_S2503_2011 = {
    "S2503_C01_001E" : 1,
    "S2503_C01_026E" : 58,
    "S2503_C01_028E" : 62,
    "S2503_C01_030E" : 67,
    "S2503_C01_032E" : 70,
    "S2503_C01_034E" : 74,
    "S2503_C01_036E" : 78,
    "S2503_C01_038E" : 82,
    "S2503_C01_040E" : 86,
    "S2503_C01_042E" : 89,
    "S2503_C01_044E" : 93,
    "GEO_ID" : 0,
    "NAME" : 319
}

index_for_variables_S2503_2010 = {
    "S2503_C01_001E" : 1,
    "S2503_C01_026E" : 59,
    "S2503_C01_028E" : 63,
    "S2503_C01_030E" : 67,
    "S2503_C01_032E" : 71,
    "S2503_C01_034E" : 75,
    "S2503_C01_036E" : 79,
    "S2503_C01_038E" : 83,
    "S2503_C01_040E" : 87,
    "S2503_C01_042E" : 91,
    "S2503_C01_044E" : 95,
    "GEO_ID" : 0,
    "NAME" : 319
}

In [97]:
S2503_data_files = ['S2503_Data_2010.json', 'S2503_Data_2011.json', 'S2503_Data_2012.json', 'S2503_Data_2013.json', 'S2503_Data_2014.json', 'S2503_Data_2015.json', 'S2503_Data_2016.json', 'S2503_Data_2017.json', 'S2503_Data_2018.json', 'S2503_Data_2019.json', 'S2503_Data_2021.json','S2503_Data_2022.json']
S2503_data = {}
for i in range(len(S2503_data_files)):
    S2503_json = json.load(open(S2503_data_files[i]))
    temp = []

    for var in S2503_json:
        if(S2503_data_files[i] == 'S2503_Data_2010.json'):
            if(var[0] != "GEO_ID"):
              temp.append([var[index_for_variables_S2503_2010['GEO_ID']],
                      var[index_for_variables_S2503_2010['NAME']],
                        int(var[index_for_variables_S2503_2010['S2503_C01_001E']]),
                        round(0.01 * float(var[index_for_variables_S2503_2010['S2503_C01_026E']]) * int(float(var[index_for_variables_S2503_2010['S2503_C01_001E']]))) if (var[index_for_variables_S2503_2010['S2503_C01_026E']] != None and float(var[index_for_variables_S2503_2010['S2503_C01_026E']]) > 0 ) else 0,
                          round(0.01 * float(var[index_for_variables_S2503_2010['S2503_C01_028E']]) * int(float(var[index_for_variables_S2503_2010['S2503_C01_001E']]))) if (var[index_for_variables_S2503_2010['S2503_C01_028E']] != None and float(var[index_for_variables_S2503_2010['S2503_C01_028E']]) > 0 ) else 0,
                            round(0.01 * float(var[index_for_variables_S2503_2010['S2503_C01_030E']]) * int(float(var[index_for_variables_S2503_2010['S2503_C01_001E']]))) if (var[index_for_variables_S2503_2010['S2503_C01_030E']] != None and float(var[index_for_variables_S2503_2010['S2503_C01_030E']]) > 0 ) else 0,
                              round(0.01 * float(var[index_for_variables_S2503_2010['S2503_C01_032E']]) * int(float(var[index_for_variables_S2503_2010['S2503_C01_001E']]))) if (var[index_for_variables_S2503_2010['S2503_C01_032E']] != None and float(var[index_for_variables_S2503_2010['S2503_C01_032E']]) > 0 ) else 0,
                                round(0.01 * float(var[index_for_variables_S2503_2010['S2503_C01_034E']]) * int(float(var[index_for_variables_S2503_2010['S2503_C01_001E']]))) if (var[index_for_variables_S2503_2010['S2503_C01_034E']] != None and float(var[index_for_variables_S2503_2010['S2503_C01_034E']]) > 0 ) else 0,
                                  round(0.01 * float(var[index_for_variables_S2503_2010['S2503_C01_036E']]) * int(float(var[index_for_variables_S2503_2010['S2503_C01_001E']]))) if (var[index_for_variables_S2503_2010['S2503_C01_036E']] != None and float(var[index_for_variables_S2503_2010['S2503_C01_036E']]) > 0 ) else 0,
                                    round(0.01 * float(var[index_for_variables_S2503_2010['S2503_C01_038E']]) * int(float(var[index_for_variables_S2503_2010['S2503_C01_001E']]))) if (var[index_for_variables_S2503_2010['S2503_C01_038E']] != None and float(var[index_for_variables_S2503_2010['S2503_C01_038E']]) > 0 ) else 0,
                                      round(0.01 * float(var[index_for_variables_S2503_2010['S2503_C01_040E']]) * int(float(var[index_for_variables_S2503_2010['S2503_C01_001E']]))) if (var[index_for_variables_S2503_2010['S2503_C01_040E']] != None and float(var[index_for_variables_S2503_2010['S2503_C01_040E']]) > 0 ) else 0,
                                        round(0.01 * float(var[index_for_variables_S2503_2010['S2503_C01_042E']]) * int(float(var[index_for_variables_S2503_2010['S2503_C01_001E']]))) if (var[index_for_variables_S2503_2010['S2503_C01_042E']] != None and float(var[index_for_variables_S2503_2010['S2503_C01_042E']]) > 0 ) else 0,
                                          round(0.01 * float(var[index_for_variables_S2503_2010['S2503_C01_044E']]) * int(float(var[index_for_variables_S2503_2010['S2503_C01_001E']]))) if (var[index_for_variables_S2503_2010['S2503_C01_044E']] != None and float(var[index_for_variables_S2503_2010['S2503_C01_044E']]) > 0 ) else 0,
                                            round(0.01 * (float(var[index_for_variables_S2503_2010['S2503_C01_026E']]) + float(var[index_for_variables_S2503_2010['S2503_C01_030E']])) * int(float(var[index_for_variables_S2503_2010['S2503_C01_001E']]))) if (var[index_for_variables_S2503_2010['S2503_C01_026E']] != None and float(var[index_for_variables_S2503_2010['S2503_C01_030E']]) > 0 and var[index_for_variables_S2503_2010['S2503_C01_030E']] != None and float(var[index_for_variables_S2503_2010['S2503_C01_026E']]) > 0) else 0,
                                              round(0.01 * (float(var[index_for_variables_S2503_2010['S2503_C01_028E']]) + float(var[index_for_variables_S2503_2010['S2503_C01_032E']])) * int(float(var[index_for_variables_S2503_2010['S2503_C01_001E']]))) if (var[index_for_variables_S2503_2010['S2503_C01_028E']] != None and float(var[index_for_variables_S2503_2010['S2503_C01_028E']]) > 0 and var[index_for_variables_S2503_2010['S2503_C01_032E']] != None and float(var[index_for_variables_S2503_2010['S2503_C01_032E']]) > 0) else 0,
                                                round(0.01 * (float(var[index_for_variables_S2503_2010['S2503_C01_034E']]) + float(var[index_for_variables_S2503_2010['S2503_C01_038E']])) * int(float(var[index_for_variables_S2503_2010['S2503_C01_001E']]))) if (var[index_for_variables_S2503_2010['S2503_C01_034E']] != None and float(var[index_for_variables_S2503_2010['S2503_C01_034E']]) > 0 and var[index_for_variables_S2503_2010['S2503_C01_038E']] != None and float(var[index_for_variables_S2503_2010['S2503_C01_038E']]) > 0) else 0,
                                                  round(0.01 * (float(var[index_for_variables_S2503_2010['S2503_C01_036E']]) + float(var[index_for_variables_S2503_2010['S2503_C01_040E']])) * int(float(var[index_for_variables_S2503_2010['S2503_C01_001E']]))) if (var[index_for_variables_S2503_2010['S2503_C01_036E']] != None and float(var[index_for_variables_S2503_2010['S2503_C01_036E']]) > 0 and var[index_for_variables_S2503_2010['S2503_C01_040E']] != None and float(var[index_for_variables_S2503_2010['S2503_C01_040E']]) > 0) else 0])    
            else:
                  temp.append([var[index_for_variables_S2503_2010['GEO_ID']],
                      var[index_for_variables_S2503_2010['NAME']],
                        var[index_for_variables_S2503_2010['S2503_C01_001E']],
                        var[index_for_variables_S2503_2010['S2503_C01_026E']],
                          var[index_for_variables_S2503_2010['S2503_C01_028E']],
                            var[index_for_variables_S2503_2010['S2503_C01_030E']],
                              var[index_for_variables_S2503_2010['S2503_C01_032E']],
                                var[index_for_variables_S2503_2010['S2503_C01_034E']],
                                  var[index_for_variables_S2503_2010['S2503_C01_036E']],
                                    var[index_for_variables_S2503_2010['S2503_C01_038E']],
                                      var[index_for_variables_S2503_2010['S2503_C01_040E']],
                                        var[index_for_variables_S2503_2010['S2503_C01_042E']],
                                          var[index_for_variables_S2503_2010['S2503_C01_044E']],
                                            "less than 35k income, less 20 percent",
                                              "less than 35k income, 30 percent or more",
                                                "35k to 75k income, 20 percent or less",
                                                  "35k to 75k income, 30 percent or more"])
        elif(S2503_data_files[i] == 'S2503_Data_2011.json'):
          if(var[0] != "GEO_ID"):
              temp.append([var[index_for_variables_S2503_2011['GEO_ID']],
                      var[index_for_variables_S2503_2011['NAME']],
                        int(var[index_for_variables_S2503_2011['S2503_C01_001E']]),
                        round(0.01 * float(var[index_for_variables_S2503_2011['S2503_C01_026E']]) * int(float(var[index_for_variables_S2503_2011['S2503_C01_001E']]))) if (var[index_for_variables_S2503_2011['S2503_C01_026E']] != None and float(var[index_for_variables_S2503_2011['S2503_C01_026E']]) > 0 ) else 0,
                          round(0.01 * float(var[index_for_variables_S2503_2011['S2503_C01_028E']]) * int(float(var[index_for_variables_S2503_2011['S2503_C01_001E']]))) if (var[index_for_variables_S2503_2011['S2503_C01_028E']] != None and float(var[index_for_variables_S2503_2011['S2503_C01_028E']]) > 0 ) else 0,
                            round(0.01 * float(var[index_for_variables_S2503_2011['S2503_C01_030E']]) * int(float(var[index_for_variables_S2503_2011['S2503_C01_001E']]))) if (var[index_for_variables_S2503_2011['S2503_C01_030E']] != None and float(var[index_for_variables_S2503_2011['S2503_C01_030E']]) > 0 ) else 0,
                              round(0.01 * float(var[index_for_variables_S2503_2011['S2503_C01_032E']]) * int(float(var[index_for_variables_S2503_2011['S2503_C01_001E']]))) if (var[index_for_variables_S2503_2011['S2503_C01_032E']] != None and float(var[index_for_variables_S2503_2011['S2503_C01_032E']]) > 0 ) else 0,
                                round(0.01 * float(var[index_for_variables_S2503_2011['S2503_C01_034E']]) * int(float(var[index_for_variables_S2503_2011['S2503_C01_001E']]))) if (var[index_for_variables_S2503_2011['S2503_C01_034E']] != None and float(var[index_for_variables_S2503_2011['S2503_C01_034E']]) > 0 ) else 0,
                                  round(0.01 * float(var[index_for_variables_S2503_2011['S2503_C01_036E']]) * int(float(var[index_for_variables_S2503_2011['S2503_C01_001E']]))) if (var[index_for_variables_S2503_2011['S2503_C01_036E']] != None and float(var[index_for_variables_S2503_2011['S2503_C01_036E']]) > 0 ) else 0,
                                    round(0.01 * float(var[index_for_variables_S2503_2011['S2503_C01_038E']]) * int(float(var[index_for_variables_S2503_2011['S2503_C01_001E']]))) if (var[index_for_variables_S2503_2011['S2503_C01_038E']] != None and float(var[index_for_variables_S2503_2011['S2503_C01_038E']]) > 0 ) else 0,
                                      round(0.01 * float(var[index_for_variables_S2503_2011['S2503_C01_040E']]) * int(float(var[index_for_variables_S2503_2011['S2503_C01_001E']]))) if (var[index_for_variables_S2503_2011['S2503_C01_040E']] != None and float(var[index_for_variables_S2503_2011['S2503_C01_040E']]) > 0 ) else 0,
                                        round(0.01 * float(var[index_for_variables_S2503_2011['S2503_C01_042E']]) * int(float(var[index_for_variables_S2503_2011['S2503_C01_001E']]))) if (var[index_for_variables_S2503_2011['S2503_C01_042E']] != None and float(var[index_for_variables_S2503_2011['S2503_C01_042E']]) > 0 ) else 0,
                                          round(0.01 * float(var[index_for_variables_S2503_2011['S2503_C01_044E']]) * int(float(var[index_for_variables_S2503_2011['S2503_C01_001E']]))) if (var[index_for_variables_S2503_2011['S2503_C01_044E']] != None and float(var[index_for_variables_S2503_2011['S2503_C01_044E']]) > 0 ) else 0,
                                            round(0.01 * (float(var[index_for_variables_S2503_2011['S2503_C01_026E']]) + float(var[index_for_variables_S2503_2011['S2503_C01_030E']])) * int(float(var[index_for_variables_S2503_2011['S2503_C01_001E']]))) if (var[index_for_variables_S2503_2011['S2503_C01_026E']] != None and float(var[index_for_variables_S2503_2011['S2503_C01_030E']]) > 0 and var[index_for_variables_S2503_2011['S2503_C01_030E']] != None and float(var[index_for_variables_S2503_2011['S2503_C01_026E']]) > 0) else 0,
                                              round(0.01 * (float(var[index_for_variables_S2503_2011['S2503_C01_028E']]) + float(var[index_for_variables_S2503_2011['S2503_C01_032E']])) * int(float(var[index_for_variables_S2503_2011['S2503_C01_001E']]))) if (var[index_for_variables_S2503_2011['S2503_C01_028E']] != None and float(var[index_for_variables_S2503_2011['S2503_C01_028E']]) > 0 and var[index_for_variables_S2503_2011['S2503_C01_032E']] != None and float(var[index_for_variables_S2503_2011['S2503_C01_032E']]) > 0) else 0,
                                                round(0.01 * (float(var[index_for_variables_S2503_2011['S2503_C01_034E']]) + float(var[index_for_variables_S2503_2011['S2503_C01_038E']])) * int(float(var[index_for_variables_S2503_2011['S2503_C01_001E']]))) if (var[index_for_variables_S2503_2011['S2503_C01_034E']] != None and float(var[index_for_variables_S2503_2011['S2503_C01_034E']]) > 0 and var[index_for_variables_S2503_2011['S2503_C01_038E']] != None and float(var[index_for_variables_S2503_2011['S2503_C01_038E']]) > 0) else 0,
                                                  round(0.01 * (float(var[index_for_variables_S2503_2011['S2503_C01_036E']]) + float(var[index_for_variables_S2503_2011['S2503_C01_040E']])) * int(float(var[index_for_variables_S2503_2011['S2503_C01_001E']]))) if (var[index_for_variables_S2503_2011['S2503_C01_036E']] != None and float(var[index_for_variables_S2503_2011['S2503_C01_036E']]) > 0 and var[index_for_variables_S2503_2011['S2503_C01_040E']] != None and float(var[index_for_variables_S2503_2011['S2503_C01_040E']]) > 0) else 0])    
          else:
                  temp.append([var[index_for_variables_S2503_2011['GEO_ID']],
                      var[index_for_variables_S2503_2011['NAME']],
                        var[index_for_variables_S2503_2011['S2503_C01_001E']],
                        var[index_for_variables_S2503_2011['S2503_C01_026E']],
                          var[index_for_variables_S2503_2011['S2503_C01_028E']],
                            var[index_for_variables_S2503_2011['S2503_C01_030E']],
                              var[index_for_variables_S2503_2011['S2503_C01_032E']],
                                var[index_for_variables_S2503_2011['S2503_C01_034E']],
                                  var[index_for_variables_S2503_2011['S2503_C01_036E']],
                                    var[index_for_variables_S2503_2011['S2503_C01_038E']],
                                      var[index_for_variables_S2503_2011['S2503_C01_040E']],
                                        var[index_for_variables_S2503_2011['S2503_C01_042E']],
                                          var[index_for_variables_S2503_2011['S2503_C01_044E']],
                                            "less than 35k income, less 20 percent",
                                              "less than 35k income, 30 percent or more",
                                                "35k to 75k income, 20 percent or less",
                                                  "35k to 75k income, 30 percent or more"])
                          
        elif(S2503_data_files[i] in ['S2503_Data_2012.json', 'S2503_Data_2013.json', 'S2503_Data_2014.json', 'S2503_Data_2015.json', 'S2503_Data_2016.json']):
            if (var[0] != "GEO_ID"): 
              temp.append([var[index_for_variables_S2503['GEO_ID']],
                    var[index_for_variables_S2503['NAME']],
                        int(var[index_for_variables_S2503['S2503_C01_001E']]),
                          round(0.01 * float(var[index_for_variables_S2503['S2503_C01_026E']]) * int(float(var[index_for_variables_S2503['S2503_C01_001E']]))) if (var[index_for_variables_S2503['S2503_C01_026E']] != None and float(var[index_for_variables_S2503['S2503_C01_026E']]) > 0) else 0,
                            round(0.01 * float(var[index_for_variables_S2503['S2503_C01_028E']]) * int(float(var[index_for_variables_S2503['S2503_C01_001E']]))) if (var[index_for_variables_S2503['S2503_C01_028E']] != None and float(var[index_for_variables_S2503['S2503_C01_028E']]) > 0) else 0,
                              round(0.01 * float(var[index_for_variables_S2503['S2503_C01_030E']]) * int(float(var[index_for_variables_S2503['S2503_C01_001E']]))) if (var[index_for_variables_S2503['S2503_C01_030E']] != None and float(var[index_for_variables_S2503['S2503_C01_030E']]) > 0) else 0,
                                round(0.01 * float(var[index_for_variables_S2503['S2503_C01_032E']]) * int(float(var[index_for_variables_S2503['S2503_C01_001E']]))) if (var[index_for_variables_S2503['S2503_C01_032E']] != None and float(var[index_for_variables_S2503['S2503_C01_032E']]) > 0) else 0,
                                  round(0.01 * float(var[index_for_variables_S2503['S2503_C01_034E']]) * int(float(var[index_for_variables_S2503['S2503_C01_001E']]))) if (var[index_for_variables_S2503['S2503_C01_034E']] != None) and float(var[index_for_variables_S2503['S2503_C01_034E']]) > 0 else 0, 
                                    round(0.01 * float(var[index_for_variables_S2503['S2503_C01_036E']]) * int(float(var[index_for_variables_S2503['S2503_C01_001E']]))) if (var[index_for_variables_S2503['S2503_C01_036E']] != None and float(var[index_for_variables_S2503['S2503_C01_036E']]) > 0) else 0,
                                      round(0.01 * float(var[index_for_variables_S2503['S2503_C01_038E']]) * int(float(var[index_for_variables_S2503['S2503_C01_001E']]))) if (var[index_for_variables_S2503['S2503_C01_038E']] != None and float(var[index_for_variables_S2503['S2503_C01_038E']]) > 0) else 0,
                                        round(0.01 * float(var[index_for_variables_S2503['S2503_C01_040E']]) * int(float(var[index_for_variables_S2503['S2503_C01_001E']]))) if (var[index_for_variables_S2503['S2503_C01_040E']] != None and float(var[index_for_variables_S2503['S2503_C01_040E']]) > 0) else 0,
                                          round(0.01 * float(var[index_for_variables_S2503['S2503_C01_042E']]) * int(float(var[index_for_variables_S2503['S2503_C01_001E']]))) if (var[index_for_variables_S2503['S2503_C01_042E']] != None and float(var[index_for_variables_S2503['S2503_C01_042E']]) > 0) else 0,
                                            round(0.01 * float(var[index_for_variables_S2503['S2503_C01_044E']]) * int(float(var[index_for_variables_S2503['S2503_C01_001E']]))) if (var[index_for_variables_S2503['S2503_C01_044E']] != None and float(var[index_for_variables_S2503['S2503_C01_044E']]) > 0) else 0,
                                              round(0.01 * (float(var[index_for_variables_S2503['S2503_C01_026E']]) + float(var[index_for_variables_S2503['S2503_C01_030E']])) * int(float(var[index_for_variables_S2503['S2503_C01_001E']]))) if (var[index_for_variables_S2503['S2503_C01_026E']] != None and float(var[index_for_variables_S2503['S2503_C01_030E']]) > 0 and var[index_for_variables_S2503['S2503_C01_030E']] != None and float(var[index_for_variables_S2503['S2503_C01_026E']]) > 0) else 0,
                                                round(0.01 * (float(var[index_for_variables_S2503['S2503_C01_028E']]) + float(var[index_for_variables_S2503['S2503_C01_032E']])) * int(float(var[index_for_variables_S2503['S2503_C01_001E']]))) if (var[index_for_variables_S2503['S2503_C01_028E']] != None and float(var[index_for_variables_S2503['S2503_C01_028E']]) > 0 and var[index_for_variables_S2503['S2503_C01_032E']] != None and float(var[index_for_variables_S2503['S2503_C01_032E']]) > 0) else 0,
                                                  round(0.01 * (float(var[index_for_variables_S2503['S2503_C01_034E']]) + float(var[index_for_variables_S2503['S2503_C01_038E']])) * int(float(var[index_for_variables_S2503['S2503_C01_001E']]))) if (var[index_for_variables_S2503['S2503_C01_034E']] != None and float(var[index_for_variables_S2503['S2503_C01_034E']]) > 0 and var[index_for_variables_S2503['S2503_C01_038E']] != None and float(var[index_for_variables_S2503['S2503_C01_038E']]) > 0) else 0,
                                                    round(0.01 * (float(var[index_for_variables_S2503['S2503_C01_036E']]) + float(var[index_for_variables_S2503['S2503_C01_040E']])) * int(float(var[index_for_variables_S2503['S2503_C01_001E']]))) if (var[index_for_variables_S2503['S2503_C01_036E']] != None and float(var[index_for_variables_S2503['S2503_C01_036E']]) > 0 and var[index_for_variables_S2503['S2503_C01_040E']] != None and float(var[index_for_variables_S2503['S2503_C01_040E']]) > 0) else 0])                                                           
            else:
                  temp.append([var[index_for_variables_S2503['GEO_ID']],
                      var[index_for_variables_S2503['NAME']],
                       var[index_for_variables_S2503['S2503_C01_001E']],
                        var[index_for_variables_S2503['S2503_C01_026E']],
                          var[index_for_variables_S2503['S2503_C01_028E']],
                            var[index_for_variables_S2503['S2503_C01_030E']],
                              var[index_for_variables_S2503['S2503_C01_032E']],
                                var[index_for_variables_S2503['S2503_C01_034E']],
                                  var[index_for_variables_S2503['S2503_C01_036E']],
                                    var[index_for_variables_S2503['S2503_C01_038E']],
                                      var[index_for_variables_S2503['S2503_C01_040E']],
                                        var[index_for_variables_S2503['S2503_C01_042E']],
                                          var[index_for_variables_S2503['S2503_C01_044E']],
                                            "less than 35k income, less 20 percent",
                                              "less than 35k income, 30 percent or more",
                                                "35k to 75k income, 20 percent or less",
                                                  "35k to 75k income, 30 percent or more"])
    
        else:
            if (var[0] != "GEO_ID"): 
              temp.append([var[index_for_variables_S2503['GEO_ID']],
                    var[index_for_variables_S2503['NAME']],
                      int(var[index_for_variables_S2503['S2503_C01_001E']]),
                      int(var[index_for_variables_S2503['S2503_C01_026E']]),
                        int(var[index_for_variables_S2503['S2503_C01_028E']]),
                          int(var[index_for_variables_S2503['S2503_C01_030E']]),
                            int(var[index_for_variables_S2503['S2503_C01_032E']]),
                              int(var[index_for_variables_S2503['S2503_C01_034E']]),
                                int(var[index_for_variables_S2503['S2503_C01_036E']]),
                                  int(var[index_for_variables_S2503['S2503_C01_038E']]),
                                    int(var[index_for_variables_S2503['S2503_C01_040E']]),
                                      int(var[index_for_variables_S2503['S2503_C01_042E']]),
                                        int(var[index_for_variables_S2503['S2503_C01_044E']]),
                                           int(var[index_for_variables_S2503['S2503_C01_026E']]) + int(var[index_for_variables_S2503['S2503_C01_030E']]) ,
                                             int(var[index_for_variables_S2503['S2503_C01_028E']]) + int(var[index_for_variables_S2503['S2503_C01_032E']]),
                                               int(var[index_for_variables_S2503['S2503_C01_034E']]) + int(var[index_for_variables_S2503['S2503_C01_038E']]),
                                                 int(var[index_for_variables_S2503['S2503_C01_036E']]) + int(var[index_for_variables_S2503['S2503_C01_040E']])])
            else: 
                temp.append([var[index_for_variables_S2503['GEO_ID']],
                      var[index_for_variables_S2503['NAME']],
                       var[index_for_variables_S2503['S2503_C01_001E']],
                        var[index_for_variables_S2503['S2503_C01_026E']],
                          var[index_for_variables_S2503['S2503_C01_028E']],
                            var[index_for_variables_S2503['S2503_C01_030E']],
                              var[index_for_variables_S2503['S2503_C01_032E']],
                                var[index_for_variables_S2503['S2503_C01_034E']],
                                  var[index_for_variables_S2503['S2503_C01_036E']],
                                    var[index_for_variables_S2503['S2503_C01_038E']],
                                      var[index_for_variables_S2503['S2503_C01_040E']],
                                        var[index_for_variables_S2503['S2503_C01_042E']],
                                          var[index_for_variables_S2503['S2503_C01_044E']],
                                            "less than 35k income, less 20 percent",
                                              "less than 35k income, 30 percent or more",
                                                "35k to 75k income, 20 percent or less",
                                                  "35k to 75k income, 30 percent or more"])
    
    S2503_data[re.search('Data_(.+?).json', S2503_data_files[i]).group(1)] = temp

# Populating the Zillow data

In [98]:
us_states = [
    ["AL", "Alabama"],
    ["AK", "Alaska"],
    ["AZ", "Arizona"],
    ["AR", "Arkansas"],
    ["CA", "California"],
    ["CO", "Colorado"],
    ["CT", "Connecticut"],
    ["DE", "Delaware"],
    ["FL", "Florida"],
    ["GA", "Georgia"],
    ["HI", "Hawaii"],
    ["ID", "Idaho"],
    ["IL", "Illinois"],
    ["IN", "Indiana"],
    ["IA", "Iowa"],
    ["KS", "Kansas"],
    ["KY", "Kentucky"],
    ["LA", "Louisiana"],
    ["ME", "Maine"],
    ["MD", "Maryland"],
    ["MA", "Massachusetts"],
    ["MI", "Michigan"],
    ["MN", "Minnesota"],
    ["MS", "Mississippi"],
    ["MO", "Missouri"],
    ["MT", "Montana"],
    ["NE", "Nebraska"],
    ["NV", "Nevada"],
    ["NH", "New Hampshire"],
    ["NJ", "New Jersey"],
    ["NM", "New Mexico"],
    ["NY", "New York"],
    ["NC", "North Carolina"],
    ["ND", "North Dakota"],
    ["OH", "Ohio"],
    ["OK", "Oklahoma"],
    ["OR", "Oregon"],
    ["PA", "Pennsylvania"],
    ["RI", "Rhode Island"],
    ["SC", "South Carolina"],
    ["SD", "South Dakota"],
    ["TN", "Tennessee"],
    ["TX", "Texas"],
    ["UT", "Utah"],
    ["VT", "Vermont"],
    ["VA", "Virginia"],
    ["WA", "Washington"],
    ["WV", "West Virginia"],
    ["WI", "Wisconsin"],
    ["WY", "Wyoming"]
]

def replace_state_abbreviation(state):
    for abbreviation, full_name in us_states:
        if state == abbreviation:
            return full_name
    return state

In [99]:
for state_abbr in zhvi['StateName'].unique():  
    state_name = replace_state_abbreviation(state_abbr)
    state_filtered_data = zhvi[zhvi['StateName'] == state_abbr]
    county_names = state_filtered_data['RegionName'].unique()
  #  print(state_name)
 #   print(len(county_names))
#    print(county_names)


In [100]:
cross_walk = pd.read_csv('CountyCrossWalk_Zillow.csv')


In [101]:
from neo4j.debug import watch
from neo4j import GraphDatabase

#watch("neo4j", out=sys.stdout) #Output debug to stdout 
#watch("neo4j", out=open('debugLogs.txt', 'w')) #Output debug to logfile 

URI = "bolt://localhost:7687"
AUTH = ("neo4j", "13781378")
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query('CREATE (:Root {name: $name})',
                            name='root',
                            database_="neo4j")
    driver.execute_query("CREATE (:Year {year: $year})",
                            year=2010,
                            database_="neo4j")
    driver.execute_query("CREATE (:Year {year: $year})",
                    year=2011,
                    database_="neo4j")
    driver.execute_query("CREATE (:Year {year: $year})",
                    year=2012,
                    database_="neo4j")
    driver.execute_query("CREATE (:Year {year: $year})",
                    year=2013,
                    database_="neo4j")
    driver.execute_query("CREATE (:Year {year: $year})",
                    year=2014,
                    database_="neo4j")
    driver.execute_query("CREATE (:Year {year: $year})",
                    year=2015,
                    database_="neo4j")
    driver.execute_query("CREATE (:Year {year: $year})",
                    year=2016,
                    database_="neo4j")
    driver.execute_query("CREATE (:Year {year: $year})",
                    year=2017,
                    database_="neo4j")
    driver.execute_query("CREATE (:Year {year: $year})",
                    year=2018,
                    database_="neo4j")
    driver.execute_query("CREATE (:Year {year: $year})",
                    year=2019,
                    database_="neo4j")
    driver.execute_query("CREATE (:Year {year: $year})",
                    year=2021,
                    database_="neo4j")
    driver.execute_query("CREATE (:Year {year: $year})",
                    year=2022,
                    database_="neo4j")

In [102]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query("""MATCH (root:Root), (y2010:Year {year:$year_2010}) , (y2011:Year {year:$year_2011}), (y2012:Year {year:$year_2012}), (y2013:Year{year:$year_2013}), (y2014:Year{year:$year_2014}), (y2015:Year{year:$year_2015}), (y2016:Year{year:$year_2016}), (y2017:Year{year:$year_2017}), (y2018:Year{year:$year_2018}), (y2019:Year{year:$year_2019}), (y2021:Year{year:$year_2021}), (y2022:Year{year:$year_2022})
                          CREATE (root)-[:HAS_YEAR]->(y2010), (root)-[:HAS_YEAR]->(y2011), (root)-[:HAS_YEAR]->(y2012), (root)-[:HAS_YEAR]->(y2013), (root)-[:HAS_YEAR]->(y2014), (root)-[:HAS_YEAR]->(y2015), (root)-[:HAS_YEAR]->(y2016), (root)-[:HAS_YEAR]->(y2017), (root)-[:HAS_YEAR]->(y2018), (root)-[:HAS_YEAR]->(y2019), (root)-[:HAS_YEAR]->(y2021), (root)-[:HAS_YEAR]->(y2022)""",
                          year_2010=2010, year_2011=2011, year_2012=2012, year_2013=2013, year_2014=2014, year_2015=2015, year_2016=2016, year_2017=2017, year_2018=2018, year_2019=2019, year_2021=2021, year_2022=2022
                          ,database_="neo4j")

In [103]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query("""CREATE TEXT INDEX state_text_name_index FOR (n:State) ON (n.name)""",database_="neo4j")
    driver.execute_query("""CREATE TEXT INDEX county_text_name_index FOR (n:County) ON (n.name)""",database_="neo4j")
    driver.execute_query("""CREATE INDEX county_fips_number FOR (n:County) ON (n.FIPS)""",database_="neo4j")
    

In [104]:
for year_data in B17017_data.keys():
    for var in B17017_data[year_data][1:]:
            if((cross_walk.FIPS==int(var[0][9:])).any()):
                cw = cross_walk[cross_walk.FIPS==int(var[0][9:])]
                #df_avg_sorted_filtered[df_avg_sorted_filtered.RegionID==cross_walk[cross_walk.FIPS==int(var[0][9:])].CountyRegionID_Zillow.values[0]]
                with GraphDatabase.driver(URI, auth=AUTH) as driver:
                    driver.execute_query("""MATCH (y:Year {year: $year})
                                        MERGE (y)-[:HAS_STATE]->(s:State {name: $sname})
                                        MERGE (s)-[:HAS_COUNTY]->(c:County {FIPS: $FIPS, name: $cname})
                                        MERGE (c)-[:HAS_POVERTY_DATA]->(p:Poverty {under_poverty_line: $under_poverty_line, above_poverty_line: $above_poverty_line})
                                        """,
                                        sname=cw.StateName.values[0], 
                                        cname=cw.CountyName.values[0],
                                        FIPS=cw.FIPS.values[0],
                                        under_poverty_line=int(var[2]) if(var[2] != None) else 0,
                                        above_poverty_line=int(var[3]) if(var[2] != None) else 0,
                                        year=int(year_data),
                                        database_="neo4j"
                )
        

In [105]:
for year_data in S2503_data.keys():
    for var in S2503_data[year_data][1:]:
            if((cross_walk.FIPS==int(var[0][9:])).any()):
                cw = cross_walk[cross_walk.FIPS==int(var[0][9:])]
                with GraphDatabase.driver(URI, auth=AUTH) as driver:
                    driver.execute_query("""MATCH (y:Year {year: $year})
                                        MERGE (y)-[:HAS_STATE]->(s:State {name: $sname})
                                        MERGE (s)-[:HAS_COUNTY]->(c:County {FIPS: $FIPS, name: $cname})
                                        MERGE (c)-[:HAS_INCOME_DATA]->(i:Income {less_than_35k_less_20_percent: $less_than_35k_less_20_percent, less_than_35k_30_percent_or_more: $less_than_35k_30_percent_or_more, between_35k_to_75k_20_percent_or_less: $i35k_to_75k_20_percent_or_less, between_35k_to_75k_30_percent_or_more: $i35k_to_75k_30_percent_or_more, above_75k_20_percent_or_less: $above_75k_20_percent_or_less, above_75k_30_percent_or_more: $above_75k_30_percent_or_more})
                                         """,
                                        sname=cw.StateName.values[0], 
                                        cname=cw.CountyName.values[0],
                                        FIPS=cw.FIPS.values[0],
                                        less_than_35k_less_20_percent=int(var[13]) if(var[13] != None) else 0,
                                        less_than_35k_30_percent_or_more=int(var[14]) if(var[14] != None) else 0,
                                        i35k_to_75k_20_percent_or_less=int(var[15]) if(var[15] != None) else 0,
                                        i35k_to_75k_30_percent_or_more=int(var[16]) if(var[16] != None) else 0,
                                        above_75k_20_percent_or_less=int(var[9]) if(var[9] != None) else 0,
                                        above_75k_30_percent_or_more=int(var[10]) if(var[10] != None) else 0,
                                        year=int(year_data),
                                        database_="neo4j"
                )

In [106]:
for year_data in S2502_data.keys():
    for var in S2502_data[year_data][1:]:
            if((cross_walk.FIPS==int(var[0][9:])).any()):
                cw = cross_walk[cross_walk.FIPS==int(var[0][9:])]
                with GraphDatabase.driver(URI, auth=AUTH) as driver:
                    driver.execute_query("""MATCH (y:Year {year: $year})
                                        MERGE (y)-[:HAS_STATE]->(s:State {name: $sname})
                                        MERGE (s)-[:HAS_COUNTY]->(c:County {FIPS: $FIPS, name: $cname})
                                        MERGE (c)-[:HAS_AGE_DATA]->(a:Age {under_35:$under_35, between_35_to_64:$between_35_to_64, above_65:$above_65})
                                        MERGE (c)-[:HAS_ETHINICITY_DATA]->(e:Ethinicity {two_or_more_races:$two_or_more, hispanic_or_latino:$hispanic, white:$white})
                                        MERGE (c)-[:HAS_EDUCATION_DATA]->(ed:Education {less_than_high_school:$less_than_high_school, high_school:$high_school, some_college:$some_college, bachelors_or_higher:$bachelors})
                                         """,
                                        sname=cw.StateName.values[0], 
                                        cname=cw.CountyName.values[0],
                                        FIPS=cw.FIPS.values[0],
                                        under_35=int(var[6]) if(var[6] != None) else 0,
                                        between_35_to_64=int(var[7]) if(var[7] != None) else 0 + int(var[8]) if(var[8] != None) else 0 + int(var[9]) if(var[9] != None) else 0,
                                        above_65=int(var[10]) if(var[10] != None) else 0 + int(var[11]) if(var[11] != None) else 0 + int(var[12]) if(var[12] != None) else 0,
                                        two_or_more=int(var[3]) if(var[3] != None) else 0,
                                        hispanic=int(var[4]) if(var[4] != None) else 0,
                                        white=int(var[5]) if(var[5] != None) else 0,
                                        less_than_high_school=int(var[13]) if(var[13] != None) else 0,
                                        high_school=int(var[14]) if(var[14] != None) else 0,
                                        some_college=int(var[15]) if(var[15] != None) else 0,
                                        bachelors=int(var[16]) if(var[16] != None) else 0,
                                        year=int(year_data),
                                        database_="neo4j")

In [107]:
years = {'2010':2010, '2011':2011, '2012':2012, '2013':2013, '2014':2014, '2015':2015, '2016':2016, '2017':2017, '2018':2018, '2019':2019, '2021':2021, '2022':2022}
for index, row in df_avg_sorted_filtered.iterrows():
    cw = cross_walk[cross_walk.CountyRegionID_Zillow==int(row['RegionID'])]
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        for year in years.keys():
            if(pd.isna(row[year]) == False):
                driver.execute_query("""MATCH (y:Year {year: $year}), (y)-[:HAS_STATE]->(s:State {name: $sname}), (s)-[:HAS_COUNTY]->(c:County {FIPS: $FIPS, name: $cname})
                                MERGE (c)-[:HAS_ZHVI_DATA]->(z:Zhvi {zhvi: $zhvi})
                             """,
                            sname=cw.StateName.values[0], 
                            cname=cw.CountyName.values[0],
                            FIPS=cw.FIPS.values[0],
                            zhvi=row[year],
                            year=years[year],
                            database_="neo4j")

In [113]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query("""MATCH (y:Year), (y)-[:HAS_STATE]->(s:State), (s)-[:HAS_COUNTY]->(c:County), (y2:Year where y2.year=y.year+1), (y2)-[:HAS_STATE]->(s2:State), (s2)-[:HAS_COUNTY]->(c2:County where c2.FIPS=c.FIPS)
                            MERGE (c)-[:HAS_NEXT_YEAR_DATA]-(c2)
                         """,
                        database_="neo4j")